In [2]:
# Import the os package, used to set env variables and check paths
import os
# import the dotenv package for client keys in .env file
from dotenv import load_dotenv
# for random UUID keys
# https://docs.python.org/3/library/uuid.html
import uuid
# Import square 
from square.client import Client
# for pretty print
import pprint
# for datetime stamp
import datetime
# for JSON
import json

In [3]:
# Get the current working directory
cwd = os.getcwd()

# Construct the .env file path
env_path = os.path.join(cwd, '.env')

# Load the .env file
load_dotenv(dotenv_path=env_path)

True

In [4]:
PROJECT_ID = os.environ['GCP_PROJ_ID']  # @param {type:"string"}
# Notice the project name is uppercase. However, the project ID is lowercase
print(PROJECT_ID)

jams-devpost


In [5]:
from google.cloud import storage
from google.oauth2 import service_account
    

# Test to see that environment can find the app credentials.
# DO NOT COMMIT TO GIT WITH THIS CELL RUN AS A PRINT    
# DO NOT COMMIT TO GIT WITH THIS CELL RUN AS A PRINT    
# DO NOT COMMIT TO GIT WITH THIS CELL RUN AS A PRINT    
# DO NOT COMMIT TO GIT WITH THIS CELL RUN AS A PRINT    
with open(os.environ['GOOGLE_APPLICATION_CREDENTIALS'], 'r') as source:
    info = json.load(source)

#with open(os.environ['GOOGLE_APPLICATION_CREDENTIALS'], 'r') as f:
#    contents = f.read()
#    info = json.load(f)
#    print(contents)

credentials = service_account.Credentials.from_service_account_info(info)
    
storage_client = storage.Client(project=PROJECT_ID, credentials=credentials)

In [6]:
from google.cloud import aiplatform

PROJECT_ID = os.environ['GCP_PROJ_ID']  # @param {type:"string"}
# Notice the project name is uppercase. However, the project ID is lowercase
print(PROJECT_ID)
# staging_bucket='jams_bucket')
aiplatform.init(project=PROJECT_ID, 
              location="us-central1",
              credentials=credentials)
          

jams-devpost


In [7]:
# test the gcp vertex AI
from vertexai.language_models import TextGenerationModel

In [8]:
# load the model
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [9]:
# Generate descriptive text for the ad for Dr. Doom
prompt = "Generate one sentence describing the marvel action figure Dr. Doom"

dr_doom_descr = generation_model.predict(prompt=prompt, max_output_tokens=180).text
print(dr_doom_descr)

is a Marvel action figure that is 6 inches tall and has 16 points of articulation.


In [10]:
# Generate descriptive text for the ad for Nick Fury
prompt = "Generate in 10 words or less a blurb describing the marvel action figure Nick Fury from the 1980s comic book"

nick_fury_descr = generation_model.predict(prompt=prompt, max_output_tokens=180).text
print(nick_fury_descr)

Nick Fury is a cigar-chomping, eye-patch-wearing, no-nonsense spy.


In [100]:
# init the square API
# using generic os environment this fails.
# However, when using the dotenv package it works
sq_client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='production'  
)
pp = pprint.PrettyPrinter(indent=4)


# Create a new catalog item and associate with a new image

In terms of the Square API

* A catalog item example would be *t-shirt*
* A catalog item variation would be *size large t-shirt"

## step 1 create a catalog item and add to library

Everytime this is called, it creates a new one in the library.

Use `delete_catalog_object()` to delete them based upon obj_id

In [101]:
# create a key
uniq_uuid = uuid.uuid4()
# convert to string
idempotency_key = str(uniq_uuid)
#print("uniq_uuid: ", uniq_uuid)
#print("idempotency_key: ", idempotency_key)

# Specify the Catalog Item
# NOTE: The first catalog item requires one instance of a variation
item_name="Action Figure"
item_abbreviation = "ACFIGURE"
item_description = "6 inch action figure"
# item variation data includes
# description and price in us cents
item_variation_name = "Dr Doom Action Figure " + dr_doom_descr
item_variation_amount = 375




result = sq_client.catalog.upsert_catalog_object(
  body = {
    "idempotency_key": idempotency_key,
    "object": {
      "type": "ITEM",
      "id": "#thisvaries1",
      "item_data": {
        "name": item_name,
        "description": item_description,
        "abbreviation": item_abbreviation,
        "variations": [
          {
            "type": "ITEM_VARIATION",
            "id": "#thisvaries2",
            "item_variation_data": {
              "item_id": "#thisvaries1",
              "name": item_variation_name,
              "pricing_type": "FIXED_PRICING",
              "price_money": {
                "amount": item_variation_amount,
                "currency": "USD"
              }
            }
          },
        ]
      }
    }
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'catalog_object': {'type': 'ITEM', 'id': '6TRVMNPU4WHUSJIM7PGOCPEW', 'updated_at': '2023-10-01T19:06:55.461Z', 'created_at': '2023-10-01T19:06:55.461Z', 'version': 1696187215461, 'is_deleted': False, 'present_at_all_locations': True, 'item_data': {'name': 'Action Figure', 'description': '6 inch action figure', 'abbreviation': 'ACFIGURE', 'is_taxable': True, 'variations': [{'type': 'ITEM_VARIATION', 'id': 'JDZFUMAN323BTBR663YQLNES', 'updated_at': '2023-10-01T19:06:55.461Z', 'created_at': '2023-10-01T19:06:55.461Z', 'version': 1696187215461, 'is_deleted': False, 'present_at_all_locations': True, 'item_variation_data': {'item_id': '6TRVMNPU4WHUSJIM7PGOCPEW', 'name': 'Dr Doom Action Figure is a Marvel action figure that is 6 inches tall and has 16 points of articulation.', 'ordinal': 0, 'pricing_type': 'FIXED_PRICING', 'price_money': {'amount': 375, 'currency': 'USD'}, 'sellable': True, 'stockable': True}}], 'product_type': 'REGULAR', 'description_html': '<p>6 inch action figure</p>', 'de

In [102]:
# print and save the object id
pp.pprint(result.body)

# Save the id
#cat_obj_item_id = result.body['catalog_object']['id']
#print(cat_obj_item_id)

{   'catalog_object': {   'created_at': '2023-10-01T19:06:55.461Z',
                          'id': '6TRVMNPU4WHUSJIM7PGOCPEW',
                          'is_deleted': False,
                          'item_data': {   'abbreviation': 'ACFIGURE',
                                           'description': '6 inch action '
                                                          'figure',
                                           'description_html': '<p>6 inch '
                                                               'action '
                                                               'figure</p>',
                                           'description_plaintext': '6 inch '
                                                                    'action '
                                                                    'figure',
                                           'is_archived': False,
                                           'is_taxable': True,
                       

In [103]:
# print and save the object id
#pp.pprint(result.body)

#
# Save the id's for catalog and variation id
#
# Note this is the catalog item id.
# Later we want to add a picture for the item variation id.
cat_obj_item_id = result.body['catalog_object']['id']
#print(cat_obj_item_id)

cat_obj_item_variations = result.body['catalog_object']['item_data']['variations']
#pp.pprint(cat_obj_item_variations)


for x in  cat_obj_item_variations:
    #pp.pprint(x)
    #pp.pprint(x['item_variation_data']['item_id'])
    if item_variation_name == x['item_variation_data']['name']:
        #pp.pprint(x)
        cat_obj_item_variation_id = x['id']
        #pp.pprint(cat_obj_item_variation_id)

print("cat_obj_item_id : ", cat_obj_item_id )
print("cat_obj_item_variation_id: ", cat_obj_item_variation_id)
# Save off this variation id so we can use to update inventory
dr_doom_variation_id=cat_obj_item_variation_id


# save for later
#item_ids = [e["item_variations"]["item_id"] for e in cat_obj_item_id]


cat_obj_item_id :  6TRVMNPU4WHUSJIM7PGOCPEW
cat_obj_item_variation_id:  JDZFUMAN323BTBR663YQLNES


## Step 2 upload an image an associate with the catalog item

Add an image to the catablog item variation.  In the case of a *large T-Shirt*, this would be an image of a Large T-Shirt.

In [104]:
# create a key
uniq_uuid = uuid.uuid4()
# convert to string
idempotency_key = str(uniq_uuid)
#print("uniq_uuid: ", uniq_uuid)
#print("idempotency_key: ", idempotency_key)


# DR DOOM
file_to_upload_path = "./sample_imgs/drdoom.jpeg" # Modify this to point to your desired file.
image_name = "Dr Doom Doll Image"
image_caption = "Dr Doom Doll"

f_stream = open(file_to_upload_path, "rb")



result = sq_client.catalog.create_catalog_image(
  request = {
    "idempotency_key": idempotency_key,
     "object_id": cat_obj_item_variation_id,
    "image": {
      "type": "IMAGE",
      "id": "#image_id",
      "image_data": {
        "name": image_name,
        "caption": image_caption
      }
    }
  },
  image_file = f_stream
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)


{'image': {'type': 'IMAGE', 'id': 'RQI77ZMUETRDFCO4NDXKPJ45', 'updated_at': '2023-10-01T19:08:21.367Z', 'created_at': '2023-10-01T19:08:21.367Z', 'version': 1696187301367, 'is_deleted': False, 'present_at_all_locations': True, 'image_data': {'name': 'Dr Doom Doll Image', 'url': 'https://items-images-production.s3.us-west-2.amazonaws.com/files/0a980973e5af640e65bc451ff2f4c565a2819d9e/original.jpeg', 'caption': 'Dr Doom Doll'}}}


# Create another variation for the same catalog item


In [105]:
# create a key
uniq_uuid = uuid.uuid4()
# convert to string
idempotency_key = str(uniq_uuid)
#print("uniq_uuid: ", uniq_uuid)
#print("idempotency_key: ", idempotency_key)

# Specify the Catalog Item
# NOTE: The first catalog item requires one instance of a variation
#item_name="Action Figure"
#item_abbreviation = "ACFIGURE"
#item_description = "something we want vertex ai to generate"
# item variation data includes
# description and price in us cents
item_variation_name = "Nick Fury Action Figure " + nick_fury_descr
item_variation_amount = 350




result = sq_client.catalog.upsert_catalog_object(
  body = {
    "idempotency_key": idempotency_key,
    "object": {
      "type": "ITEM_VARIATION",
      "id": "#new",
      "item_variation_data": {
        "item_id": cat_obj_item_id,
        "name": item_variation_name,
        "pricing_type": "FIXED_PRICING",
        "price_money": {
          "amount": item_variation_amount,
          "currency": "USD"
        }
      }
    }
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)







{'catalog_object': {'type': 'ITEM_VARIATION', 'id': '55E3MQD2DCZXVX44XW6HCEMV', 'updated_at': '2023-10-01T19:08:32.572Z', 'created_at': '2023-10-01T19:08:32.572Z', 'version': 1696187312572, 'is_deleted': False, 'present_at_all_locations': True, 'item_variation_data': {'item_id': '6TRVMNPU4WHUSJIM7PGOCPEW', 'name': 'Nick Fury Action Figure Nick Fury is a cigar-chomping, eye-patch-wearing, no-nonsense spy.', 'pricing_type': 'FIXED_PRICING', 'price_money': {'amount': 350, 'currency': 'USD'}, 'sellable': True, 'stockable': True}}, 'id_mappings': [{'client_object_id': '#new', 'object_id': '55E3MQD2DCZXVX44XW6HCEMV'}]}


In [106]:
# print and save the object id
pp.pprint(result.body)

#
# Save the id's for catalog and variation id
#
# Note this is the catalog item id.
# Later we want to add a picture for the item variation id.
# NOTE: I was expecting to have to search through the variations
# to find the variation id, when you have two items, but it appears you get just the last item
# so we don't have to search.  We know where it is.
cat_obj_item_variation_id = result.body['catalog_object']['id']

print(cat_obj_item_variation_id)

# Save off this variation id so we can use to update inventory
nick_fury_variation_id=cat_obj_item_variation_id



{   'catalog_object': {   'created_at': '2023-10-01T19:08:32.572Z',
                          'id': '55E3MQD2DCZXVX44XW6HCEMV',
                          'is_deleted': False,
                          'item_variation_data': {   'item_id': '6TRVMNPU4WHUSJIM7PGOCPEW',
                                                     'name': 'Nick Fury Action '
                                                             'Figure Nick Fury '
                                                             'is a '
                                                             'cigar-chomping, '
                                                             'eye-patch-wearing, '
                                                             'no-nonsense spy.',
                                                     'price_money': {   'amount': 350,
                                                                        'currency': 'USD'},
                                                     'pricing_type': 'FIXED_PRI

# Add an image for the variation

In [107]:
# create a key
uniq_uuid = uuid.uuid4()
# convert to string
idempotency_key = str(uniq_uuid)
#print("uniq_uuid: ", uniq_uuid)
print("idempotency_key: ", idempotency_key)


# Nick Fury
file_to_upload_path = "./sample_imgs/nickfury.jpeg" # Modify this to point to your desired file.
image_name = "Nick Fury Doll Image"
image_caption = "Nick Fury Doll"

f_stream = open(file_to_upload_path, "rb")

result = sq_client.catalog.create_catalog_image(
   request = {
     "idempotency_key": idempotency_key,
      "object_id": cat_obj_item_variation_id,
     "image": {
       "type": "IMAGE",
       "id": "#image_id",
       "image_data": {
         "name": image_name,
         "caption": image_caption
       }
     }
   },
   image_file = f_stream
 )

if result.is_success():
   print(result.body)
elif result.is_error():
   print(result.errors)


idempotency_key:  515db70b-acee-4fbe-a7fe-b862cd534a07
{'image': {'type': 'IMAGE', 'id': 'DKBWMTYIE4JN5YRY72SDQUQJ', 'updated_at': '2023-10-01T19:08:54.396Z', 'created_at': '2023-10-01T19:08:54.396Z', 'version': 1696187334396, 'is_deleted': False, 'present_at_all_locations': True, 'image_data': {'name': 'Nick Fury Doll Image', 'url': 'https://items-images-production.s3.us-west-2.amazonaws.com/files/2e7be50da564218c0327c39300dfd2198445bd2d/original.jpeg', 'caption': 'Nick Fury Doll'}}}


# update the inventory count (This is a required api usage for the hackathon)

This will update the current count.  It is not an adjustment.

Example: if you currently have 10 items, you use this call to specify 15 items, afterwards inventory is 15 items.
There is another call which is "inventory adjustment" that will increase/decrease the inventory account 
by the specified amount.  For this reason, simply use the api to specify a count.

In [108]:
# Save off this variation id so we can use to update inventory

# print the id's of interest
print("action figure cat id: ", cat_obj_item_id)
print("dr doom var id: ", dr_doom_variation_id)
print("nick fury var id: ", nick_fury_variation_id)


action figure cat id:  6TRVMNPU4WHUSJIM7PGOCPEW
dr doom var id:  JDZFUMAN323BTBR663YQLNES
nick fury var id:  55E3MQD2DCZXVX44XW6HCEMV


# update inventory for Dr Doom Variation 

In [109]:
# create a key
uniq_uuid = uuid.uuid4()
#print("uniq_uuid: ", uniq_uuid)

# constants between inventory updates
online_store_loc_id = "LZBNSDTSHAMM8"
quantity_update_number = "10"  # assume we want to specify a particular count, not adj by this amount.

# get current timestamp as ISO8601 yyyy-mm-ddThr:min:sec.sssZ
current_time = datetime.datetime.now().isoformat(timespec='seconds')
current_time = current_time + 'Z'
time_of_received_inventory = current_time


# Dr doom Inventory
variation_id = dr_doom_variation_id

result = sq_client.inventory.batch_change_inventory(
  body = {
    "idempotency_key": str(uniq_uuid),
    "changes": [
      {
        "type": "PHYSICAL_COUNT",
        "physical_count": {
          "catalog_object_id": variation_id,
          "state": "IN_STOCK",
          "location_id": online_store_loc_id,
          "quantity": quantity_update_number,
          "occurred_at": time_of_received_inventory
        }
      }
    ]
  }
)

# If the existing count is 15, and the update is to have
# 15, then the results will be {}.
#
# If the existing count is 10 and the update is 15,
# then the results will have some json

print("results:")
print(result)
if result.is_success():
  print("success")
  print(result.body)
elif result.is_error():
  print("failure")
  print(result.errors)

results:
<ApiResponse {"counts":[{"catalog_object_id":"JDZFUMAN323BTBR663YQLNES","catalog_object_type":"ITEM_VARIATION","state":"IN_STOCK","location_id":"LZBNSDTSHAMM8","quantity":"10","calculated_at":"2023-10-01T19:10:10.595Z"}]}>
success
{'counts': [{'catalog_object_id': 'JDZFUMAN323BTBR663YQLNES', 'catalog_object_type': 'ITEM_VARIATION', 'state': 'IN_STOCK', 'location_id': 'LZBNSDTSHAMM8', 'quantity': '10', 'calculated_at': '2023-10-01T19:10:10.595Z'}]}


# update nick fury variation inventory count

In [110]:
# create a key
uniq_uuid = uuid.uuid4()
#print("uniq_uuid: ", uniq_uuid)

# constants between inventory updates
online_store_loc_id = "LZBNSDTSHAMM8"
quantity_update_number = "5"  # assume we want to specify a particular count, not adj by this amount.

# get current timestamp as ISO8601 yyyy-mm-ddThr:min:sec.sssZ
current_time = datetime.datetime.now().isoformat(timespec='seconds')
current_time = current_time + 'Z'
time_of_received_inventory = current_time


# Nick Fury Inventory
variation_id = nick_fury_variation_id

result = sq_client.inventory.batch_change_inventory(
  body = {
    "idempotency_key": str(uniq_uuid),
    "changes": [
      {
        "type": "PHYSICAL_COUNT",
        "physical_count": {
          "catalog_object_id": variation_id,
          "state": "IN_STOCK",
          "location_id": online_store_loc_id,
          "quantity": quantity_update_number,
          "occurred_at": time_of_received_inventory
        }
      }
    ]
  }
)

# If the existing count is 15, and the update is to have
# 15, then the results will be {}.
#
# If the existing count is 10 and the update is 15,
# then the results will have some json

print("results:")
print(result)
if result.is_success():
  print("success")
  print(result.body)
elif result.is_error():
  print("failure")
  print(result.errors)

results:
<ApiResponse {"counts":[{"catalog_object_id":"55E3MQD2DCZXVX44XW6HCEMV","catalog_object_type":"ITEM_VARIATION","state":"IN_STOCK","location_id":"LZBNSDTSHAMM8","quantity":"5","calculated_at":"2023-10-01T19:10:14.359Z"}]}>
success
{'counts': [{'catalog_object_id': '55E3MQD2DCZXVX44XW6HCEMV', 'catalog_object_type': 'ITEM_VARIATION', 'state': 'IN_STOCK', 'location_id': 'LZBNSDTSHAMM8', 'quantity': '5', 'calculated_at': '2023-10-01T19:10:14.359Z'}]}


# Get info about a particular item

In [90]:
# catalog item 
object_id = cat_obj_item_id

# nick fury variation item
#object_id = nick_fury_variation_id

# dr doom variation item
#object_id = dr_doom_variation_id


result = sq_client.catalog.retrieve_catalog_object(
  object_id = object_id
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'object': {'type': 'ITEM', 'id': 'N66OBPI7PJY6DKHPIU4LWBGA', 'updated_at': '2023-10-01T18:58:11.725Z', 'created_at': '2023-10-01T18:57:50.069Z', 'version': 1696186691725, 'is_deleted': False, 'present_at_all_locations': True, 'item_data': {'name': 'Action Figure', 'description': '6 inch action figure', 'abbreviation': 'ACFIGURE', 'is_taxable': True, 'variations': [{'type': 'ITEM_VARIATION', 'id': 'OH2NI7SEWGALMSTERIO2TJZ4', 'updated_at': '2023-10-01T18:58:10.675Z', 'created_at': '2023-10-01T18:57:50.069Z', 'version': 1696186690675, 'is_deleted': False, 'present_at_all_locations': True, 'item_variation_data': {'item_id': 'N66OBPI7PJY6DKHPIU4LWBGA', 'name': 'Dr Doom Action Figure is a Marvel action figure that is 6 inches tall and has 16 points of articulation.', 'ordinal': 0, 'pricing_type': 'FIXED_PRICING', 'price_money': {'amount': 375, 'currency': 'USD'}, 'location_overrides': [{'location_id': 'LZBNSDTSHAMM8', 'track_inventory': True}], 'sellable': True, 'stockable': True, 'image_id

In [29]:
pp.pprint(result.body)

{   'object': {   'created_at': '2023-10-01T17:25:03.923Z',
                  'id': '7DYIBAXRC6DA4FA2MBLHY5SZ',
                  'is_deleted': False,
                  'item_data': {   'abbreviation': 'ACFIGURE',
                                   'delivery_fulfillment_preferences_id': 'fprefs_127siquj0a3oo1afthgaishjx',
                                   'description': 'something we want vertex ai '
                                                  'to generate',
                                   'description_html': '<p>something we want '
                                                       'vertex ai to '
                                                       'generate</p>',
                                   'description_plaintext': 'something we want '
                                                            'vertex ai to '
                                                            'generate',
                                   'ecom_available': True,
                     

# add visibility to the items so they appear in the online 

This does not appear to be doable via API.  To do manually:

0. Vist the online [site](https://rtp-gcp-usergroup.square.site/s/shop) and it should not list any items for sale.
1. Open `Square Dashboard`
2. Select `Item Library``
3. Click the text title for the action figure.  eg. the item category item.
4. The `Edit Item`  will appear.
5. Scroll down to `Square Online settings`
6. Click the slider next to rtp gcp user group
7. Vist the online [site](https://rtp-gcp-usergroup.square.site/s/shop) and it should be listed online. 


Here is the [Sites API](https://developer.squareup.com/docs/sites-api/use-the-api), it does not look like it has an API to do this.  Instead it appears to be for listing the sites and not updating items visibility.

Perhaps you can use the upsert API to do this, but I can't figure it out.  It seems you have to update everything all over again.  I just want to change the visibility....

# modify category for items
ie. modify these items to be "Marvel Comics".  Create some items using "lord of the rings.", Modify these items to be of type "lord of the rings".

I gave up trying to change the category or the online visibility via the API.

## Delete a catalog object item

`DONT RUN UNLESS YOU WANT TO RECREATE THE ITEM`

In [19]:
delete_id = "S5IJI4PUFA5VTB2Y6NDZ6HQV"
result = sq_client.catalog.delete_catalog_object(
  object_id = delete_id
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'deleted_object_ids': ['45KACZNXRA7PM32CAGXGRZA4', 'S5IJI4PUFA5VTB2Y6NDZ6HQV'], 'deleted_at': '2023-09-30T13:12:08.572Z'}
